In [11]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
from getpass import getpass
import os
from dotenv import load_dotenv

In [22]:
load_dotenv(dotenv_path='.streamlit\secrets.toml')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')
OPENAI_API_KEY  = os.getenv('OPENAI')

## Setting up the tools

In [16]:
# Define which tools the agent can use to answer user queries
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

## Prompt template

This instructs the agent on what to do. Generally, the template should incorporate:

- **tools**: which tools the agent has access and how and when to call them.
- **intermediate_steps**: These are tuples of previous (AgentAction, Observation) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.
- **input**: generic user input

In [17]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""

In [18]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]
    
    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [19]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

## Output parser

The output parser is responsible for parsing the LLM output into AgentAction and AgentFinish. This usually depends heavily on the prompt used.

This is where you can change the parsing to do retries, handle whitespace, etc.


In [20]:
class CustomOutputParser(AgentOutputParser):
    
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [21]:
output_parser = CustomOutputParser()

In [23]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0)

In [24]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [25]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=tool_names
)

In [27]:
dict_thing = {'input': 'I meant interactively via a GUI', 'chat_history': [AIMessage(content='How can I help you?', additional_kwargs={}, example=False), HumanMessage(content='Hello', additional_kwargs={}, example=False), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, example=False), HumanMessage(content='What does pop do in the following code?\n\n# Set up a prompt template\nclass CustomPromptTemplate(BaseChatPromptTemplate):\n    # The template to use\n    template: str\n    # The list of tools available\n    tools: List[Tool]\n    \n    def format_messages(self, **kwargs) -> str:\n        # Get the intermediate steps (AgentAction, Observation tuples)\n        # Format them in a particular way\n        intermediate_steps = kwargs.pop("intermediate_steps")\n        thoughts = ""\n        for action, observation in intermediate_steps:\n            thoughts += action.log\n            thoughts += f"\\nObservation: {observation}\\nThought: "\n        # Set the agent_scratchpad variable to that value\n        kwargs["agent_scratchpad"] = thoughts\n        # Create a tools variable from the list of tools provided\n        kwargs["tools"] = "\\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])\n        # Create a list of tool names for the tools provided\n        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])\n        formatted = self.template.format(**kwargs)\n        return [HumanMessage(content=formatted)]', additional_kwargs={}, example=False), AIMessage(content='The `pop()` function is used to remove and return the last item from a list. It modifies the list in-place.', additional_kwargs={}, example=False), HumanMessage(content='What would be some code to create a basic plot in matplotlib interactively?', additional_kwargs={}, example=False), AIMessage(content='To create a basic plot in matplotlib interactively, you can use the following code:\n\n```python\nimport matplotlib.pyplot as plt\n\n# Create some data\nx = [1, 2, 3, 4, 5]\ny = [2, 4, 6, 8, 10]\n\n# Plot the data\nplt.plot(x, y)\n\n# Show the plot\nplt.show()\n```', additional_kwargs={}, example=False), HumanMessage(content='I meant interactively via a GUI', additional_kwargs={}, example=False), AIMessage(content='To create a basic plot in matplotlib interactively via a GUI, you can use the following code:\n\n```python\nimport matplotlib.pyplot as plt\n\n# Create some data\nx = [1, 2, 3, 4, 5]\ny = [2, 4, 6, 8, 10]\n\n# Create a figure and axes\nfig, ax = plt.subplots()\n\n# Plot the data\nax.plot(x, y)\n\n# Show the plot interactively\nplt.show(block=True)\n```', additional_kwargs={}, example=False)], 'output': 'To create a basic plot in matplotlib interactively via a GUI, you can use the following code:\n\n```python\nimport matplotlib.pyplot as plt\n\n# Create some data\nx = [1, 2, 3, 4, 5]\ny = [2, 4, 6, 8, 10]\n\n# Create a figure and axes\nfig, ax = plt.subplots()\n\n# Plot the data\nax.plot(x, y)\n\n# Show the plot interactively\nplt.show(block=True)\n```', 'intermediate_steps': [(AgentAction(tool='_Exception', tool_input='Invalid or incomplete response', log='Could not parse LLM output: To create a basic plot in matplotlib interactively via a GUI, you can use the following code:\n\n```python\nimport matplotlib.pyplot as plt\n\n# Create some data\nx = [1, 2, 3, 4, 5]\ny = [2, 4, 6, 8, 10]\n\n# Create a figure and axes\nfig, ax = plt.subplots()\n\n# Plot the data\nax.plot(x, y)\n\n# Show the plot interactively\nplt.show(block=True)\n```\n\nThis code will display the plot in a separate window and allow you to interact with it using the GUI provided by matplotlib.'), 'Invalid or incomplete response')]}

NameError: name 'AIMessage' is not defined

In [43]:
from langchain.schema import (
    AgentAction,
    AgentFinish,
    BaseOutputParser,
    BasePromptTemplate,
    OutputParserException,
)

x = 'e'
def testnig(x):
    try:
        1/x
    except OutputParserException as e:
        print('The error is:', e)

In [44]:
testnig(x)

TypeError: unsupported operand type(s) for /: 'int' and 'str'